## Fluid dynamic model for PV module temperature estimation

<img src="sourcedata/PV_module_temp.png"><br>
Source: DelftX training; delftx.tudelft.nl

In [1]:
# Meteorological Variables
Gm = 400 # W/m2
w = 5 # wind sped (m/s)
Tgr = 13+273 # Temperature of ground (Kelvin)
Ta = 15+273 # Temperature of air (Kelvin)

# PV Module Parameters
L = 1.56 # [m] PV module Length   
W = 1.05 # [m] PV module Width
emi_top = 0.84 # PV module top surface emissivity 
emi_bottom = 0.893 # PV module back surface emissivity
Tnoct = 273+43 # NOCT Temperature is 43 degree celcius, obtained from the module spec sheet.
Ref = 0.1 # reflectivity of PV module surface
Eff_stc = 0.20 # efficiecny of the PV module at STC

#Physical Constants
g = 9.8 # [m/s2] 
Pr = 0.71 # Prandle Number
kvis = 14.6e-6 # air viscosity 
k = 0.026 # [W/m*K] thermal conductivity of air  in   
SB = 5.67e-8 # [W*m^-2*K^-4 ] Stefan-Boltzmnan constant 

In [5]:
Tm = 293  # [K] - Initial Temperature of the PV module --> 20+273 = 293 K
for i in range(5):
    # sky temperature (since we have 6 okta it can be assumed to be equal to air temp)
    Tsky = Ta

    # calculation of radiative heat transfer coefficient between module and sky
    h_r_sky = emi_top * SB * (Tm**2 + Tsky**2) * (Tm + Tsky)

    # calculation of radiative heat transfer coefficient between module and ground
    h_r_gr = emi_bottom * SB * (Tm**2 + Tgr**2) * (Tm + Tgr)
    
    # since wind speed is greater than 3 m/s we have turbulent flow for forced convection

    # hydraulic diameter of a PV module
    Dh = 2*L*W/(L+W)

    # Reynold's number
    Re = w*Dh/kvis

    # air density
    ro = 1.225 # [kg/m3]

    # air specific heat
    cv = 713 # [J/kgK]

    h_forced = 0.028*Re**-0.2/(Pr**0.4)*ro*cv*w

    h_f2 = w**0.8
    
    # free convection

    # Grashof number
    Gr = g*1/Tm*(Tm - Ta)*Dh**3/kvis**2

    # Nusselt number
    Nu = 0.21*(Gr * Pr)**0.32

    # free convection HTC
    h_free = Nu*k/Dh
    
    # total convection for front surface
    h_c_top = (h_free**3 + h_forced**3)**(1/3)
    
    # absorptivity 
    alpha = (1 - Ref)*(1 - Eff_stc)
    
    # temperature INOCT for rack mounted version
    Tinoct = Tnoct-3

    # R is the Ratio of actual heat transfer coefficient (back side) to the ideal heat transfer coefficient (top side)
    R = (alpha*Gm-h_c_top*(Tinoct - Ta) - emi_top*SB*(Tinoct**4 - Tsky**4)) / \
    (h_c_top*(Tinoct - Ta) + emi_top*SB*(Tinoct**4 - Tsky**4))
    
    # HTC for bottom side
    h_c_bottom = R * h_c_top
    
    # total HTC
    h_c = h_c_top + h_c_bottom
    
    # module temperature
    Tm = (alpha*Gm + h_c*Ta + h_r_sky*Tsky + h_r_gr*Tgr) / (h_c + h_r_sky + h_r_gr)

    print(h_r_sky, h_r_gr, h_f2, h_free, h_c_top, R, h_c, Tm)

4.670817615323999 4.9147755226605 3.623898318388478 3.4035055118009003 10.594185471213006 -0.2696248666642787 7.737729626120561 304.05756890537566
4.94587919251282 5.205880874490061 3.623898318388478 4.885683928450847 10.818658419340768 -0.2798735080761084 7.790802534842741 303.4709360313151
4.930938743006334 5.190067943743905 3.623898318388478 4.830827511855203 10.80758477480276 -0.279374673660351 7.78821930528566 303.5014998546541
4.93171617623362 5.190890774098951 3.623898318388478 4.83372363907378 10.808163724016937 -0.2794007707035409 7.788354449636552 303.4999060213642
4.931675632238226 5.190847862585841 3.623898318388478 4.833572718203076 10.808133538629201 -0.2793994100965868 7.788347403691067 303.499989132178


In [6]:
Tm - 273

30.499989132177973

In [4]:
# since wind speed is greater than 3 m/s we have turbulent flow for forced convection

# hydraulic diameter of a PV module
Dh = 2*L*W/(L+W)

# Reynold's number
Re = w*Dh/kvis

# air density
ro = 1.225 # [kg/m3]

# air specific heat
cv = 713 # [J/kgK]

h_forced = 0.028*Re**-0.2/(Pr**0.4)*ro*cv*w

h_f2 = w**0.8

(h_forced, h_f2)

(10.475776106212734, 3.623898318388478)